In [1]:
from typing import Annotated,TypedDict,Sequence
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage
from langchain_core.messages import ToolMessage
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph,END
from langgraph.prebuilt import ToolNode

load_dotenv()

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

@tool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

@tool
def substract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

tools=[add,substract,multiply]

model=ChatOpenAI(model="gpt-4o").bind_tools(tools)

def model_call(state: AgentState)->AgentState:
    system_message=SystemMessage(content="You are a helpful AI assistant. You can use tools to help you with your tasks.")
    response=model.invoke([system_message]+state["messages"])
    return {"messages":[response]}


def should_continue(state: AgentState):
    message=state["messages"]
    last_message=message[-1]
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"

graph=StateGraph(AgentState)
graph.add_node("our_agent",model_call)

tool_node=ToolNode(tools=tools)
graph.add_node("tools",tool_node)

graph.set_entry_point("our_agent")

graph.add_conditional_edges(
    "our_agent",
    should_continue,
    {
        "continue":"tools",
        "end":END
    }
)

graph.add_edge("tools","our_agent") 

app=graph.compile()


def print_stream(stream):
    for s in stream:
        message=s["messages"][-1]
        if isinstance(message,tuple):
            print(message)
        else:
            message.pretty_print()


inputs={"messages":[("user","What is 2+2?")]}
print_stream(app.stream(inputs,stream_mode="values"))


================================ Human Message =================================

What is 2+2?


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-xxxxx***********xxxx. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}